In [ ]:
%autosave 15
%matplotlib inline

import numpy as np
import scipy as scipy
import scipy.optimize as sopt
import pandas as pd
import math
import random
import scipy.stats

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [ ]:
def printAllPoints(data, classes):
    green = [[], []]
    blue = [[], []]
    for (x, y) in data:
        if (classes[(x, y)] == 0):
            green[0].append(x)
            green[1].append(y)
        else:
            blue[0].append(x)
            blue[1].append(y)
    plt.plot(green[0], green[1], 'g.', blue[0], blue[1], 'b.')

In [ ]:
def get_data(transform, name):
    fin = open('chips.txt', 'r')
    lines = fin.readlines()
    fin.close()
    
    classes = dict()
    data = list()
    
    for x, y, z in [map(float, x.split(',')) for x in lines]:
        a, b = transform(x, y)
        data.append((a, b))
        classes[(a, b)] = z
    
    return (data, classes, name, transform)

In [ ]:
def toPolar(x, y, x0, y0):
    x, y = x - x0, y - y0
    r = (x ** 2 + y ** 2) ** (1 / 2)
    a = math.atan2(x, y)
    return (r, a)

def getCenter():
    fin = open('chips.txt', 'r')
    lines = fin.readlines()
    fin.close()

    x0, y0 = 0, 0
    for x, y, z in [map(float, x.split(',')) for x in lines]:
        x0 += x
        y0 += y
    x0 = x0 / len(lines)
    y0 = y0 / len(lines)
    
    return (x0, y0)

In [ ]:
simpleData = get_data(lambda x, y:(x, y), '-')
sumData = get_data(lambda x, y:(x, x + y), 'x, y -> x, x + y')
polarData = get_data(lambda x, y: toPolar(x, y, 0, 0), 'polar(0, 0)')

center = getCenter()
polarData2 = get_data(lambda x, y: toPolar(x, y, center[0], center[1]), 'polar(center_x, center_y)')

wideData = get_data(lambda x, y: ( toPolar(x, y, center[0], center[1])[0] * 10 
                                 , toPolar(x, y, center[0], center[1])[0] + toPolar(x, y, center[0], center[1])[1]
                                 )
                    , 'wide')


data = [simpleData, sumData, polarData, polarData2, wideData]

In [ ]:
#printAllPoints(wideData[0], wideData[1])

In [ ]:
#printAllPoints(simpleData[0], simpleData[1])

In [ ]:
#printAllPoints(sumData[0], sumData[1])

In [ ]:
#printAllPoints(polarData[0], polarData[1])

In [ ]:
#printAllPoints(polarData2[0], polarData2[1])

In [ ]:
def minkowskiDistance(x, y, p):
    res = 0
    for i in range(len(x)):
        res += abs(x[i] - y[i]) ** p
    return res ** (1 / p)

# https://en.wikipedia.org/wiki/Cosine_similarity
def cosineSimilarity(x, y):
    res, a, b = 0, 0, 0
    for i in range(len(x)):
        res += x[i] * y[i]
        a += x[i] ** 2
        b += y[i] ** 2
    a = a ** (1 / 2)
    b = b ** (1 / 2)
    return res / a / b
    

metrics = [lambda x, y: minkowskiDistance(x, y, 1), 
           lambda x, y: minkowskiDistance(x, y, 2)
          ]
#lambda x, y: cosineSimilarity(x, y)]    

metric_names = dict()
metric_names[metrics[0]] = 'minkowski with p = 1'
metric_names[metrics[1]] = 'minkowski with p = 2'
#metric_names[metrics[2]] = 'cosine'

#for metric in metrics:
#    print(metric((1, 1), (2, 2)))

In [ ]:
kernels = [
    (lambda x: 1 / 2, 'uniform'),
    (lambda x: 1 - abs(x), 'triangular'),
    (lambda x: 3 / 4 * (1 - x * x), 'parabolic'),
    (lambda x: (1 - x ** 2) ** 2 * 15 / 16, 'quartic')
]

In [ ]:
def k_fold_cv(k, length):
    one_fold_length = length // k
    others = length % k
    indexies = [i for i in range(length)]
    result = list()
    for i in range(k):
        test_suit = list()
        train_suit = list()

        for j in range(one_fold_length):
            index = indexies[int(np.random.uniform(0, len(indexies))) % len(indexies)]
            test_suit.append(index)
            indexies.remove(index)
        if others > 0:
            others -= 1
            index = indexies[int(np.random.uniform(0, len(indexies))) % len(indexies)]
            test_suit.append(index)
            indexies.remove(index)
        
        for j in range(length):
            if j not in test_suit:
                train_suit.append(j)
        
        result.append((train_suit, test_suit))
    return result

In [ ]:
'''
green = [[], []]
blue = [[], []]

for x in np.arange(-1, 1.5, 0.01):
    for y in np.arange(-1, 1.5, 0.01):
        predicted = predict_class(best_neighbor, best_metric, best_data[0], best_data[1], best_data[2](x, y), best_kernel)

        if (predicted == 0):
            green[0].append(x)
            green[1].append(y)
        else:
            blue[0].append(x)
            blue[1].append(y)

plt.title(best_string)
plt.plot(green[0], green[1], 'g.', blue[0], blue[1], 'b.')
'''

In [ ]:
printAllPoints(simpleData[0], simpleData[1])

In [ ]:
def get_class(x, classes):
    temp = classes[x]
    if temp == 0:
        return -1
    else:
        return 1

def lagrange(lambdas, train_suit, classes, kernel):
    result = -sum(lambdas)
    
    for i in range(len(train_suit)):
        for j in range(len(train_suit)):
            x_i = train_suit[i]
            x_j = train_suit[j]
            result += lambdas[i] * lambdas[j] * get_class(x_i, classes) * get_class(x_j, classes) * kernel(x_i, x_j)
    return result

def get_constraint(train_suit, classes):
    def constraint(x):
        result = 0
        for i in range(len(x)):
            result += get_class(train_suit[i], classes) * x[i]
        return result
    return constraint
    
def predict_svm(point, kernel, w, w0):
    result = kernel(point, w) - w0
    if result < 0:
        return 0
    return 1

# training_suit = []
# classes = {}

def SVM(data, classes, kernel, C, cv_params = (1, 10)):

    tfold, kfold = cv_params
    kfold_index = k_fold_cv(kfold, len(data))
    max_score = 0
    TP  = [0, 0]
    FP  = [0, 0]
    ALL = [0 , 0]
    for train_suit, test_suit in kfold_index:
        training_suit = [data[i] for i in train_suit]
        testing_suit = [data[i] for i in test_suit]
        def lagrange_gradient(x, *args):
            def dv_dxi(i):
                result = -1
                for j in range(len(x)):
                    x_i = training_suit[i]
                    x_j = training_suit[j]

                    result += x[j] * get_class(x_i, classes) * get_class(x_j, classes) * kernel(x_i, x_j)
                return result
            return [dv_dxi(i) for i in range(len(x))]


        lambdas = sopt.minimize(lagrange, 
                                np.array([0 for i in range(len(training_suit))]), 
                                (training_suit, classes, kernel), 
                                jac=lagrange_gradient, 
                                bounds=[(0,C) for i in range(len(training_suit))],
                                constraints={'type':'eq', 'fun':get_constraint(training_suit, classes)})

        w = np.array([0 for i in range(len(training_suit[0]))], dtype='float64')
        for i in range(len(training_suit)):
            lambda_i = lambdas.x[i]
            y_i = get_class(training_suit[i], classes)
            x_i = training_suit[i]
            w += np.array([lambda_i * y_i * x_i[j] for j in range(len(x_i))], dtype='float64')
            
        w0 = 0
        count = 0
        for i in range(len(lambdas.x)):
            lambda_i = lambdas.x[i]
            if lambda_i > 0:
                w0 += kernel(w, training_suit[i]) - get_class(training_suit[i], classes)
                count += 1
        if count != 0:
            w0 /= count
        
        for point in testing_suit:
            predicted = int(predict_svm(point, kernel, w, w0))
            real_class = int(classes[point])
            if predicted == real_class:
                TP[predicted] += 1
            else:
                FP[predicted] += 1
            ALL[real_class] += 1
        
    if TP[1] > 0:
        recall = TP[1] / ALL[1]
        precision = TP[1] / (TP[1] + FP[1])
        # F1 measure
        return 2 * (precision * recall) / (precision + recall), (kernel, w, w0)
    return 0, (kernel, w, w0)

In [ ]:
def simple_kernel(x, y):
    return sum(np.array(x) * np.array(y))

def square_kernel(x, y):
    return simple_kernel(x, y) ** 2

def paraboloid_kernel(x, y):
    return simple_kernel([x[0], x[1], x[0]**2 + x[1]**2], [y[0], y[1], y[0]**2 + y[1]**2])

def parab(data):
    w = np.array([0, 0], dtype='float64')
    input_data, classes, transform_name, transform = data
    for point in input_data:
        w += np.array(point, dtype='float64')
    w /= len(data)
    def kernel(x, y):
        return simple_kernel([x[0], x[1], (x[0] - w[0])**2 + (x[1] - w[1])**2], [y[0], y[1], (y[0] - w[0])**2 + (y[1] - w[1])**2])
    return kernel


def paraboloid_kernel2(x, y):
    return square_kernel([x[0], x[1], x[0]**2 + x[1]**2], [y[0], y[1], y[0]**2 + y[1]**2])

def gaussianKernel(sigma=1):
    FG = lambda x : np.array([(x[0]**2 + x[1]**2) , x[0], x[1]])
    return lambda x, y: np.exp(-np.dot(FG(np.array(x) - np.array(y)),  FG(np.array(x) - np.array(y))) / (2 * (sigma ** 2)))

bikernels = [(parab(simpleData), 'square paraboloid'),
             (gaussianKernel(1), 'gaussian'),
             (simple_kernel, '<x,y>'),
             (square_kernel, '<x,y>^2'),
             (paraboloid_kernel, 'simple paraboloid'),
             ]

svm_data = [data[0], data[-2], data[-1]]

In [ ]:
current = 0
bext = None
for kfold in [5]:
    for kernel in bikernels:
        for input_data, classes, transform_name, transform in svm_data:
            for C in [0.114]:
                score, tmp = SVM(input_data, classes, kernel[0], C, cv_params=(1,kfold))
                if current < score:
                    current = score
                    bext = tmp
                print(kernel[1], transform_name, score)

In [ ]:
def knn_predict_class(point, learningSet, classes, knn_k, knnMetric, knnKernel):
    distances = [(knnMetric(lpoint, point), lpoint) for lpoint in learningSet]
    distances.sort()
    d = distances[knn_k][0]
    if (d == 0):
        d = 1
    s = [0, 0]
    for j in range(knn_k):
        dist, p = distances[j]
        s[int(classes[p])] += knnKernel(dist / d)
    if (s[0] > s[1]):
        return 0
    else:
        return 1


def getKnn(learningSet, testingSet, classes, knn_k, knnMetric, knnKernel):
    result = dict()
    for point in testingSet:
        result[point] = knn_predict_class(point, learningSet, classes, knn_k, knnMetric, knnKernel)    
    return result

In [ ]:

def SVM2(learningSet, testingSet, classes, kernel, C):
    def lagrange_gradient(x, *args):
        def dv_dxi(i):
            result = -1
            for j in range(len(x)):
                x_i = learningSet[i]
                x_j = learningSet[j]

                result += x[j] * get_class(x_i, classes) * get_class(x_j, classes) * kernel(x_i, x_j)
            return result
        
        return [dv_dxi(i) for i in range(len(x))]

    lambdas = sopt.minimize(lagrange, 
                            np.array([0 for i in range(len(learningSet))]),
                            (learningSet, classes, kernel), 
                            jac=lagrange_gradient, 
                            bounds=[(0,C) for i in range(len(learningSet))],
                            constraints={'type':'eq', 'fun':get_constraint(learningSet, classes)})

    w = np.array([0 for i in range(len(learningSet[0]))], dtype='float64')
    for i in range(len(learningSet)):
        lambda_i = lambdas.x[i]
        y_i = get_class(learningSet[i], classes)
        x_i = learningSet[i]
        w += np.array([lambda_i * y_i * x_i[j] for j in range(len(x_i))], dtype='float64')
            
    w0 = 0
    count = 0
    for i in range(len(lambdas.x)):
        lambda_i = lambdas.x[i]
        if lambda_i > 0:
            w0 += kernel(w, learningSet[i]) - get_class(learningSet[i], classes)
            count += 1
    if count != 0:
        w0 /= count
        
    res = {}
        
    for point in testingSet:
        res[point] = int(predict_svm(point, kernel, w, w0))
    
    return res

In [ ]:
def getConfusion(data, pClasses, trueClasses):
    true_positive = [0, 0]
    false_positive = [0, 0]
    
    for point in data:
        trueClass = trueClasses[point]
        predicted = pClasses[point]
        
        if predicted == trueClass:
            true_positive[predicted] += 1
        else:
            false_positive[predicted] += 1
    
    return (true_positive, false_positive)
    
    

def getF1Score(conf):
    true_positive, false_positive = conf
    all_points = [0, 0]
    all_points[0] = true_positive[0] + false_positive[1]
    all_points[1] = true_positive[1] + false_positive[0]
       
    if true_positive[1] > 0:
        recall = true_positive[1] / all_points[1]
        precision = true_positive[1] / (true_positive[1] + false_positive[1])
        return 2 * (precision * recall) / (precision + recall)
    return 0

In [ ]:
# H0: difference between predicted classes follows a symmetric distribution around zero
# H1: difference between predicted does not follow a symmetric distribution around zero.

# reference link: https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test

# two-sided, for one-sided count only + or - values
# Significance Level: 0.01 or 0.05?
def getWilcoxonRank(res1, res2):
    diff = []
    for i in range(len(res1)):
        if (res2[i] - res1[i] != 0):
            m = abs(res2[i] - res1[i])
            diff.append([m, (res2[i] - res1[i]) / m, 0])
    diff.sort()
    n = len(diff)

    for r in range(n):
        diff[r][2] = r + 1
    
    i = 0
    while (i < n):
        s = 0
        prev = i
        while ((i < n) and (diff[i][0] == diff[prev][0])):
            s += diff[i][2]
            i += 1
        if (i - prev > 1):
            for t in range(prev, i):
                diff[t][2] = s / (i - prev)
    
    ans_m = 0
    ans_p = 0
    for i in range(n):
        if (diff[i][1] * diff[i][2] < 0):
            ans_m -= diff[i][1] * diff[i][2]
        else:
            ans_p += diff[i][1] * diff[i][2]
    
    
    #z = 
    #prob = 2. * distributions.norm.sf(abs(z))
    #return (ans, z, prob)
    
    return ans_m, ans_p

In [ ]:
def getCV(data, k):
    cData = data[::]
    random.shuffle(cData)
    foldSize = len(cData) // k
    folds = [cData[i * foldSize : (i + 1) * foldSize] for i in range(k)]
    q = 0
    for i in range(k * foldSize, len(cData)):
        folds[q].append(cData[i])
    return folds

In [ ]:
#(data, classes, name, transform) for every in data = [simpleData, sumData, polarData, polarData2, wideData]

k = 12

for d, classes, name, tr in [wideData]:
    
    folds = getCV(d, k)
    
    r1, r2 = [], []
    rr1, rr2 = {}, {}
    
    for i in range(len(folds)):
        testingSet = folds[i] 
        learningSet = []
        for j in range(len(folds)):
            if (j != i):
                learningSet += folds[j]
 

       # knnClasses, svmClasses = compareMethods(d, classes, 6, 4, , , 0)

        metr = lambda x, y: minkowskiDistance(x, y, 1)
        kern = lambda x: 1 / 2
    
        knnClasses = getKnn(learningSet, testingSet, classes, 4, metr, kern)
        svmClasses = SVM2(learningSet, testingSet, classes, simple_kernel, 8)
        
        r1.append(getF1Score(getConfusion(testingSet, knnClasses, classes)))
        r2.append(getF1Score(getConfusion(testingSet, svmClasses, classes)))
        
        rr1.update(knnClasses)
        rr2.update(svmClasses)
            
    
    wr = getWilcoxonRank(r1, r2)
    print(len(r1))
    wr2 = scipy.stats.wilcoxon(r2, r1)
    print("WR:", wr)
    print("WR2:", wr2)
    
    
    print("knn f1:", getF1Score(getConfusion(d, rr1, classes)))
    print("svm f1:", getF1Score(getConfusion(d, rr2, classes)))
    
        
    
    

# можно еще f1-score и confusion-matrix посчитать
# tp, fp = getConfusion(predictedClasses)
# f1 = getF1Score((tp, fp))